# SQL Guidebook - Personal Reference Guide
# Week 7 Major Assignment

## Introduction
This guidebook demonstrates advanced SQL queries using a sample e-commerce database. 
It covers table creation, data manipulation, joins, window functions, CTEs, and more.


In [ ]:
## 1. Database Setup

import sqlite3
import pandas as pd

# Create connection to SQLite database
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

print("Database connected successfully!")

### CREATE

In [ ]:
# Create Customers Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE,
    city TEXT,
    state TEXT,
    signup_date DATE
)
''')

# Create Products Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price DECIMAL(10, 2),
    stock_quantity INTEGER
)
''')

# Create Orders Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    status TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)
''')

# Create Order Items Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS order_items (
    item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    unit_price DECIMAL(10, 2),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

conn.commit()
print("Tables created successfully!")

### INSERT

In [ ]:
# ============================================================================
# DATA INSERTION
# ============================================================================

# Insert Customers
customers_data = [
    (1, 'John', 'Doe', 'john.doe@email.com', 'New York', 'NY', '2024-01-15'),
    (2, 'Jane', 'Smith', 'jane.smith@email.com', 'Los Angeles', 'CA', '2024-02-20'),
    (3, 'Mike', 'Johnson', 'mike.j@email.com', 'Chicago', 'IL', '2024-03-10'),
    (4, 'Emily', 'Brown', 'emily.b@email.com', 'Houston', 'TX', '2024-01-25'),
    (5, 'David', 'Lee', 'david.lee@email.com', 'Phoenix', 'AZ', '2024-04-05'),
    (6, 'Sarah', 'Wilson', 'sarah.w@email.com', 'Philadelphia', 'PA', '2024-02-14'),
    (7, 'Tom', 'Martinez', 'tom.m@email.com', 'San Antonio', 'TX', '2024-03-20'),
    (8, 'Lisa', 'Anderson', 'lisa.a@email.com', 'San Diego', 'CA', '2024-01-30')
]

cursor.executemany('''
    INSERT OR IGNORE INTO customers VALUES (?, ?, ?, ?, ?, ?, ?)
''', customers_data)

# Insert Products
products_data = [
    (1, 'Laptop Pro', 'Electronics', 1299.99, 50),
    (2, 'Wireless Mouse', 'Electronics', 29.99, 200),
    (3, 'USB-C Cable', 'Accessories', 19.99, 300),
    (4, 'Desk Chair', 'Furniture', 249.99, 75),
    (5, 'Monitor 27"', 'Electronics', 399.99, 100),
    (6, 'Keyboard Mechanical', 'Electronics', 89.99, 150),
    (7, 'Desk Lamp', 'Furniture', 45.99, 120),
    (8, 'Notebook Set', 'Stationery', 12.99, 500),
    (9, 'Webcam HD', 'Electronics', 79.99, 80),
    (10, 'Phone Stand', 'Accessories', 24.99, 250)
]

cursor.executemany('''
    INSERT OR IGNORE INTO products VALUES (?, ?, ?, ?, ?)
''', products_data)

# Insert Orders
orders_data = [
    (1, 1, '2024-05-01', 1349.98, 'Completed'),
    (2, 2, '2024-05-03', 429.98, 'Completed'),
    (3, 3, '2024-05-05', 89.99, 'Completed'),
    (4, 1, '2024-05-10', 249.99, 'Completed'),
    (5, 4, '2024-05-12', 1719.96, 'Shipped'),
    (6, 5, '2024-05-15', 142.97, 'Completed'),
    (7, 2, '2024-05-18', 79.99, 'Processing'),
    (8, 6, '2024-05-20', 89.99, 'Completed'),
    (9, 7, '2024-05-22', 399.99, 'Shipped'),
    (10, 3, '2024-05-25', 474.97, 'Completed')
]

cursor.executemany('''
    INSERT OR IGNORE INTO orders VALUES (?, ?, ?, ?, ?)
''', orders_data)

# Insert Order Items
order_items_data = [
    (1, 1, 1, 1, 1299.99),
    (2, 1, 2, 1, 29.99),
    (3, 1, 3, 1, 19.99),
    (4, 2, 5, 1, 399.99),
    (5, 2, 2, 1, 29.99),
    (6, 3, 6, 1, 89.99),
    (7, 4, 4, 1, 249.99),
    (8, 5, 1, 1, 1299.99),
    (9, 5, 5, 1, 399.99),
    (10, 5, 3, 1, 19.99),
    (11, 6, 8, 10, 12.99),
    (12, 6, 3, 1, 19.99),
    (13, 7, 9, 1, 79.99),
    (14, 8, 6, 1, 89.99),
    (15, 9, 5, 1, 399.99),
    (16, 10, 7, 3, 45.99),
    (17, 10, 8, 20, 12.99),
    (18, 10, 10, 5, 24.99)
]

cursor.executemany('''
    INSERT OR IGNORE INTO order_items VALUES (?, ?, ?, ?, ?)
''', order_items_data)

conn.commit()
print("Data inserted successfully!")
print(f"Customers: {len(customers_data)}")
print(f"Products: {len(products_data)}")
print(f"Orders: {len(orders_data)}")
print(f"Order Items: {len(order_items_data)}")

In [ ]:
# ============================================================================
# QUERY 1: Basic SELECT with WHERE, ORDER BY, and LIMIT
# ============================================================================

"""
**Title:** Basic Product Filtering and Sorting

**Explanation:** 
This query retrieves products in the Electronics category, ordered by price 
from highest to lowest, limiting results to the top 5 most expensive items.

**SQL Concepts:** SELECT, FROM, WHERE, ORDER BY, LIMIT
"""

query1 = """
SELECT 
    product_name,
    category,
    price,
    stock_quantity
FROM products
WHERE category = 'Electronics'
ORDER BY price DESC
LIMIT 5
"""

result1 = pd.read_sql_query(query1, conn)
print("\n=== QUERY 1: Top 5 Most Expensive Electronics ===")
print(result1)


In [ ]:
# ============================================================================
# QUERY 2: Aggregate Functions with GROUP BY and HAVING
# ============================================================================

"""
**Title:** Sales Analysis by Customer with Aggregates

**Explanation:** 
Groups orders by customer and calculates total spending, number of orders, 
and average order value. Only shows customers with total spending over $500.

**SQL Concepts:** COUNT, SUM, AVG, GROUP BY, HAVING
"""

query2 = """
SELECT 
    customer_id,
    COUNT(*) as total_orders,
    SUM(total_amount) as total_spent,
    AVG(total_amount) as avg_order_value,
    MAX(total_amount) as largest_order
FROM orders
GROUP BY customer_id
HAVING SUM(total_amount) > 500
ORDER BY total_spent DESC
"""

result2 = pd.read_sql_query(query2, conn)
print("\n=== QUERY 2: High-Value Customers (>$500) ===")
print(result2)


In [ ]:
# ============================================================================
# QUERY 3: INNER JOIN - Combining Multiple Tables
# ============================================================================

"""
**Title:** Customer Order Details with INNER JOIN

**Explanation:** 
Joins customers and orders tables to show customer information alongside 
their order history. Only includes customers who have placed orders.

**SQL Concepts:** INNER JOIN, Table Aliases
"""

query3 = """
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name as customer_name,
    c.city,
    c.state,
    o.order_id,
    o.order_date,
    o.total_amount,
    o.status
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
ORDER BY c.customer_id, o.order_date
"""

result3 = pd.read_sql_query(query3, conn)
print("\n=== QUERY 3: Customer Orders (INNER JOIN) ===")
print(result3.head(10))

In [ ]:
# ============================================================================
# QUERY 4: LEFT JOIN - Including All Records from Left Table
# ============================================================================

"""
**Title:** All Customers with Order Count (LEFT JOIN)

**Explanation:** 
Shows all customers including those who haven't placed orders yet.
Uses LEFT JOIN to keep all customers and COUNT to show order frequency.
COALESCE handles NULL values for customers without orders.

**SQL Concepts:** LEFT JOIN, COUNT, GROUP BY, COALESCE (NULL handling)
"""

query4 = """
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name as customer_name,
    c.email,
    COUNT(o.order_id) as order_count,
    COALESCE(SUM(o.total_amount), 0) as total_spent
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.email
ORDER BY order_count DESC, total_spent DESC
"""

result4 = pd.read_sql_query(query4, conn)
print("\n=== QUERY 4: All Customers with Order Statistics ===")
print(result4)

In [ ]:
# ============================================================================
# QUERY 5: Multiple JOINs - Order Details
# ============================================================================

"""
**Title:** Complete Order Information with Multiple JOINs

**Explanation:** 
Combines four tables to show detailed order information including
customer name, product details, and quantities purchased.

**SQL Concepts:** Multiple INNER JOINs, String Concatenation
"""

query5 = """
SELECT 
    o.order_id,
    c.first_name || ' ' || c.last_name as customer_name,
    p.product_name,
    p.category,
    oi.quantity,
    oi.unit_price,
    (oi.quantity * oi.unit_price) as line_total,
    o.order_date,
    o.status
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
INNER JOIN products p ON oi.product_id = p.product_id
ORDER BY o.order_date DESC, o.order_id
"""

result5 = pd.read_sql_query(query5, conn)
print("\n=== QUERY 5: Detailed Order Information ===")
print(result5.head(10))

In [ ]:
# ============================================================================
# QUERY 6: CASE WHEN - Data Transformation
# ============================================================================

"""
**Title:** Customer Segmentation with CASE WHEN

**Explanation:** 
Categorizes customers into segments (VIP, Regular, New) based on their
total spending. Demonstrates conditional logic in SQL.

**SQL Concepts:** CASE WHEN, Conditional Logic, Subquery
"""

query6 = """
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name as customer_name,
    COALESCE(SUM(o.total_amount), 0) as total_spent,
    COUNT(o.order_id) as order_count,
    CASE 
        WHEN COALESCE(SUM(o.total_amount), 0) >= 1500 THEN 'VIP'
        WHEN COALESCE(SUM(o.total_amount), 0) >= 500 THEN 'Regular'
        ELSE 'New'
    END as customer_segment
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC
"""

result6 = pd.read_sql_query(query6, conn)
print("\n=== QUERY 6: Customer Segmentation ===")
print(result6)


In [ ]:
# ============================================================================
# QUERY 7: Window Functions - RANK and ROW_NUMBER
# ============================================================================

"""
**Title:** Product Ranking by Category with Window Functions

**Explanation:** 
Uses window functions to rank products within each category by price.
Demonstrates PARTITION BY to create separate rankings per category.

**SQL Concepts:** RANK(), ROW_NUMBER(), OVER, PARTITION BY
"""

query7 = """
SELECT 
    product_name,
    category,
    price,
    stock_quantity,
    RANK() OVER (PARTITION BY category ORDER BY price DESC) as price_rank,
    ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC) as row_num
FROM products
ORDER BY category, price_rank
"""

result7 = pd.read_sql_query(query7, conn)
print("\n=== QUERY 7: Product Rankings by Category ===")
print(result7)


In [ ]:
# ============================================================================
# QUERY 8: Common Table Expressions (CTE) with WITH
# ============================================================================

"""
**Title:** Monthly Sales Analysis using CTE

**Explanation:** 
Uses a CTE to organize monthly sales data, then calculates total revenue
and order counts per month. CTEs make complex queries more readable.

**SQL Concepts:** WITH (CTE), STRFTIME (Date function), Subquery organization
"""

query8 = """
WITH monthly_sales AS (
    SELECT 
        STRFTIME('%Y-%m', order_date) as month,
        order_id,
        total_amount,
        status
    FROM orders
)
SELECT 
    month,
    COUNT(order_id) as total_orders,
    SUM(total_amount) as monthly_revenue,
    AVG(total_amount) as avg_order_value,
    COUNT(CASE WHEN status = 'Completed' THEN 1 END) as completed_orders
FROM monthly_sales
GROUP BY month
ORDER BY month
"""

result8 = pd.read_sql_query(query8, conn)
print("\n=== QUERY 8: Monthly Sales Summary (CTE) ===")
print(result8)


In [ ]:
# ============================================================================
# QUERY 9: Advanced Window Functions - Running Total
# ============================================================================

"""
**Title:** Running Total of Sales with Window Functions

**Explanation:** 
Calculates a running total of sales ordered by date, showing cumulative
revenue over time. Demonstrates advanced window function usage.

**SQL Concepts:** SUM() OVER, Window Frame, Running Calculations
"""

query9 = """
SELECT 
    order_id,
    order_date,
    total_amount,
    SUM(total_amount) OVER (ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as running_total,
    AVG(total_amount) OVER (ORDER BY order_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as moving_avg_3
FROM orders
ORDER BY order_date
"""

result9 = pd.read_sql_query(query9, conn)
print("\n=== QUERY 9: Running Total and Moving Average ===")
print(result9)

In [ ]:
# ============================================================================
# QUERY 10: UPDATE Operation
# ============================================================================

"""
**Title:** UPDATE Product Stock After Sale

**Explanation:** 
Demonstrates updating records. This reduces stock quantity for a product
after a sale (simulated scenario).

**SQL Concepts:** UPDATE, SET, WHERE
"""

print("\n=== QUERY 10: UPDATE Product Stock ===")
print("Before update:")
before_update = pd.read_sql_query("SELECT * FROM products WHERE product_id = 2", conn)
print(before_update)

cursor.execute("""
    UPDATE products
    SET stock_quantity = stock_quantity - 5
    WHERE product_id = 2
""")
conn.commit()

print("\nAfter reducing stock by 5 units:")
after_update = pd.read_sql_query("SELECT * FROM products WHERE product_id = 2", conn)
print(after_update)


In [ ]:
# ============================================================================
# BONUS QUERY 11: UNION - Combining Results
# ============================================================================

"""
**Title:** UNION - Combining Queries for Inventory Alert

**Explanation:** 
Uses UNION to combine low stock products and high-value products into
a single result set for management review.

**SQL Concepts:** UNION, Multiple SELECT statements
"""

query11 = """
SELECT 
    product_name,
    'Low Stock' as alert_type,
    stock_quantity as metric
FROM products
WHERE stock_quantity < 100

UNION

SELECT 
    product_name,
    'High Value' as alert_type,
    price as metric
FROM products
WHERE price > 200
ORDER BY alert_type, metric DESC
"""

result11 = pd.read_sql_query(query11, conn)
print("\n=== BONUS QUERY 11: Inventory Alerts (UNION) ===")
print(result11)


In [ ]:
# ============================================================================
# Summary Statistics
# ============================================================================

print("\n" + "="*60)
print("DATABASE SUMMARY")
print("="*60)

summary_queries = {
    "Total Customers": "SELECT COUNT(*) FROM customers",
    "Total Products": "SELECT COUNT(*) FROM products",
    "Total Orders": "SELECT COUNT(*) FROM orders",
    "Total Revenue": "SELECT SUM(total_amount) FROM orders",
    "Average Order Value": "SELECT AVG(total_amount) FROM orders"
}

for label, query in summary_queries.items():
    result = cursor.execute(query).fetchone()[0]
    if "Revenue" in label or "Value" in label:
        print(f"{label}: ${result:,.2f}")
    else:
        print(f"{label}: {result}")

# Close connection
conn.close()
print("\n✅ SQL Guidebook Complete! Database connection closed.")